In [52]:
import os
os.environ['ALFRED_ROOT'] = '/root/data/home/hoyeung/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

import torch
import pprint
import json
import numpy as np
from collections import defaultdict

import spacy
nlp = spacy.load("en_core_web_sm")

In [95]:
!pip install rouge --user

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
class args:
    pass

# settings
args.seed = 123
args.data = '/root/data_alfred/json_feat_2.1.0'
args.splits = '/root/data_alfred/splits/oct21.json'
args.preprocess = False #!
args.pp_folder = 'pp'
args.save_every_epoch = False #!
args.model = 'seq2seq_im'
args.gpu = True
args.dout = 'exp/model:seq2seq_im'
args.resume = False #!

# vocab objects

In [3]:
vocab = torch.load(os.path.join(args.data, "%s.vocab" % args.pp_folder))

# splits


- Load here
https://github.com/Chucooleg/alfred/blob/6d2a6d9b210ea2ab57a3d6c6b2810f796e9ad2d1/models/train/train_seq2seq.py#L80

In [4]:
# load train/valid/tests splits
with open(args.splits) as f:
    splits = json.load(f)
    pprint.pprint({k: len(v) for k, v in splits.items()})

{'tests_seen': 1533,
 'tests_unseen': 1529,
 'train': 21023,
 'valid_seen': 820,
 'valid_unseen': 821}


In [9]:
splits['train'][0]

{'repeat_idx': 0,
 'task': 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648'}

In [31]:
train_task_set = [t for t in splits['train'] if t['repeat_idx'] == 0]
valid_seen_task_set = [t for t in splits['valid_seen'] if t['repeat_idx'] == 0]
valid_unseen_task_set = [t for t in splits['valid_unseen'] if t['repeat_idx'] == 0]

In [32]:
print(len(train_task_set))
print(len(valid_seen_task_set))
print(len(valid_unseen_task_set))

6574
251
255


# Batch

- Loaded here https://github.com/Chucooleg/alfred/blob/6d2a6d9b210ea2ab57a3d6c6b2810f796e9ad2d1/models/model/seq2seq.py#L277

In [33]:
def load_task_jsons(task):
    '''
    load all preprocessed jsons with matching task index from disk. 
    do this to gather all 3 versions of language annotations.
    '''
    dataset = []
    for i in range(3):
        json_path = os.path.join(args.data, task['task'], '%s' % args.pp_folder, 'ann_%d.json' % i)
        if os.path.exists(json_path):
            retry = 0
            while True:
                try:
                    if retry > 0:
                        print ('retrying {}'.format(retry))
                    with open(json_path) as f:
                        dataset.append(json.load(f))
                    break
                except:
                    retry += 1
                    time.sleep(5)
                    pass
    return dataset 

In [41]:
exs = load_task_jsons(train_task_set[0])
len(exs[0]['ann']['instr']) == len(exs[1]['ann']['instr']) == len(exs[2]['ann']['instr'])
num_subgoals = len(exs[0]['ann']['instr'])

True

In [89]:
exs_pos_to_tokens = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
exs_tag_to_tokens = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for ex in exs:
    
    pos_to_tokens = defaultdict(lambda: defaultdict(list))
    tag_to_tokens = defaultdict(lambda: defaultdict(list))

    for subgoal_i, subgoal in enumerate(ex['ann']['instr']):
        sent = nlp(' '.join(subgoal))
        for i, token in enumerate(sent):
            exs_pos_to_tokens[token.pos_][subgoal_i][ex['repeat_idx']].append(str(token))
            exs_tag_to_tokens[token.tag_][subgoal_i][ex['repeat_idx']].append(str(token))

In [84]:
for subgoal_i in num_subgoals:
    exs_pos_to_tokens['NOUN'][subgoal_i]

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {0: defaultdict(list,
                         {0: ['turn', 'counter'],
                          1: ['knife', 'table'],
                          2: ['slice', 'vegetable', 'table'],
                          3: ['turn', 'right', 'fridge'],
                          4: ['fridge', 'knife', 'fridge', 'fridge'],
                          5: ['turn', 'turn', 'counter'],
                          6: ['slice', 'vegetable'],
                          7: ['turn', 'turn', 'right', 'fridge'],
                          8: ['fridge',
                           'slice',
                           'knife',
                           'fridge',
                           'fridge',
                           'slice',
                           'fridge'],
                          9: ['turn',
                           'turn',
                           'turn',
                           'turn',
                           'right',

In [90]:
exs_pos_to_tokens['NOUN'][0]

defaultdict(list,
            {0: ['turn', 'counter'],
             1: ['turn', 'stool', 'walk', 'table'],
             2: ['turn', 'move', 'table']})

In [92]:
from nltk.translate.bleu_score import sentence_bleu
sentence_bleu([['turn', 'counter'], ['turn', 'stool', 'walk', 'table']], ['turn', 'move', 'table'], weights=(1,0,0,0))

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-g

0.6666666666666666

In [102]:
from rouge import Rouge 
rouge = Rouge()
scores = rouge.get_scores([['turn', 'move', 'table']], [['turn', 'stool', 'walk', 'table']])

AttributeError: 'list' object has no attribute 'split'

In [97]:
scores

NameError: name 'scores' is not defined

defaultdict(list,
            {'_SP': [' ', '  ', ' ', '  ', '  ', '  ', '  ', '  ', ' '],
             'VB': ['walk'],
             'JJ': ['straight'],
             ',': [','],
             'NN': ['turn', 'counter'],
             'VBN': ['left'],
             'TO': ['to'],
             'DT': ['the'],
             '.': ['.']})

In [70]:
pos_to_tokens

defaultdict(list,
            {'SPACE': [' ', '  ', ' ', '  ', '  ', '  ', '  ', '  ', ' '],
             'VERB': ['walk', 'left'],
             'ADJ': ['straight'],
             'PUNCT': [',', '.'],
             'NOUN': ['turn', 'counter'],
             'PART': ['to'],
             'DET': ['the']})

In [49]:
doc_pos

['SPACE',
 'VERB',
 'SPACE',
 'ADJ',
 'SPACE',
 'PUNCT',
 'SPACE',
 'NOUN',
 'SPACE',
 'VERB',
 'SPACE',
 'PART',
 'SPACE',
 'DET',
 'SPACE',
 'NOUN',
 'SPACE',
 'PUNCT']

In [ ]:
pos_cat = 'NOUN'
